# Worksheet 05

Name:  Zihan Li

UID: U83682995

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Step 1
Dataset: [0, 0.5, 1.5, 2, 6, 6.5, 7]
Initial centroids: [0, 2]

Step 2
For centroid 0, the closest points are [0, 0.5].
For centroid 2, the closest points are [1.5, 2, 6, 6.5, 7].
\
Step 3
New centroid for cluster 1 (previously centered at 0): mean([0, 0.5]) = 0.25
New centroid for cluster 2 (previously centered at 2): mean([1.5, 2, 6, 6.5, 7]) = 4.6

Step 4
With the new centroids, we repeat the assignment step.

For centroid 0.25, the closest points are now [0, 0.5, 1.5].
For centroid 4.6, the closest points are [2, 6, 6.5, 7].

Step 5
Update the centroids of each cluster again.

New centroid for cluster 1: mean([0, 0.5, 1.5]) = 0.67 (approximately)
New centroid for cluster 2: mean([2, 6, 6.5, 7]) = 5.38 (approximately)
We will repeat steps 4 and 5 (Assignment and Update) until the centroids do not change significantly, indicating that the algorithm has converged.

After recalculating, the new centroids are approximately:
New centroid for cluster 1: 0.67
New centroid for cluster 2: 5.38


b) Describe in plain english what the cost function for k means is.


The cost function for k-means measures how spread out the clusters are. It calculates the distance from each point to its cluster's center, squares these distances, and then adds them all up. The goal is to make this total as small as possible, meaning the clusters are tight and compact.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

The k-means algorithm can yield different solutions for the same number of clusters on a given dataset due to random initialization of centroids, the tendency to find local minima rather than the global minimum, varying convergence criteria, and sensitivity to data distribution and outliers.

d) Does Lloyd's Algorithm always converge? Why / why not?


Lloyd's Algorithm always converges because it continually reduces or maintains a cost measure, which has a lower limit, leading to a stable configuration where the cost can't be further reduced, although it may not be the optimal solution.

e) Follow along in class the implementation of Kmeans

In [3]:
%pip install scikit-learn
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
     --------------------------------------- 10.6/10.6 MB 59.5 MB/s eta 0:00:00
     --------------------------------------- 46.2/46.2 MB 72.5 MB/s eta 0:00:00
     ---------------------------------------- 302.2/302.2 kB ? eta 0:00:00
[[-3.25873474  1.80410328]
 [-0.50173528  0.56587087]
 [ 1.83555022 -4.10276176]
 [ 1.88512895  1.53939516]]
[[-3.14555626  1.85455619]
 [-0.29808003  0.19297643]
 [ 1.84180743 -4.03258775]
 [ 1.90104571  1.8261627 ]]
[[-3.10208548  1.85147285]
 [-0.13209903  0.06838121]
 [ 1.84126343 -4.06791444]
 [ 1.90165131  1.93255625]]
[[-3.07703402  1.85266958]
 [-0.03760357  0.01175879]
 [ 1.84585117 -4.10098602]
 [ 1.904406    1.98450725]]
[[-3.07703402  1.85266958]
 [-0.01886062  0.02039989]
 [ 1.84585117 -4.10098602]
 [ 1.91248297  2.00204407]]
[[-3.07703402  1.85266958]
 [-0.01886062  0.02039989]
 [ 1.84585117 -4.10098602]
 [ 1.91248297  2.00204407]]
